In [2]:
import pandas as pd
from pathlib import Path
import html
import preprocessor as p
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import numpy as np
from tqdm import tqdm
from emolexHelper import pandasMultiCategorySentiment

In [3]:
BEFORE_DIR = "before_pandemic"
AFTER_DIR = "since_pandemic"

In [4]:
# load all the csv files from
#before = dict()
df = pd.DataFrame(columns=["cat", "username", "text", "date", "geo", "hashtags", "tweet_id", "mentions", "permalink", "replies", "retweets", "replies_to", "mentioned_urls"])

for file in list((Path(".") / BEFORE_DIR).glob('**/*.csv')):
    #before[file.stem] = pd.read_csv(file)
    user_df = pd.read_csv(file)
    # BUG? why are there nan text rows?
    # dropping them!
    user_df = user_df.drop(user_df[pd.isnull(user_df["text"])].index)
    user_df["cat"] = BEFORE_DIR
    user_df["username"] = file.stem
    df = df.append(user_df)
    #before[file.stem] = before[file.stem].drop(before[file.stem][pd.isnull(before[file.stem]["text"])].index)

#after = dict()
for file in list((Path(".") / AFTER_DIR).glob('**/*.csv')):
    #after[file.stem] = pd.read_csv(file)
    user_df = pd.read_csv(file)
    # BUG? why are there nan text rows?
    # dropping them!
    user_df = user_df.drop(user_df[pd.isnull(user_df["text"])].index)
    user_df["cat"] = AFTER_DIR
    user_df["username"] = file.stem
    df = df.append(user_df)
    #after[file.stem] = after[file.stem].drop(after[file.stem][pd.isnull(after[file.stem]["text"])].index)

In [5]:
# before.keys()
# before["1baldchick"]["text"]

In [6]:
#stopwords

In [7]:
def decodeAndClean(s:str):
    out = html.unescape(s)
    out = p.clean(out)
    return out

In [8]:
analyzer = SentimentIntensityAnalyzer()
def getSentiment(s):
    global analyzer
    
    try:
        out = analyzer.polarity_scores(s)["compound"]
    except Exception:
        print(s)
    return out

In [9]:
import nltk
import string
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
nltk.download('stopwords')

def preprocess(text):
    text = text.lower()
    
    text_p = "".join([char for char in text if char not in string.punctuation])
    
    words = word_tokenize(text_p)
    
    stop_words = stopwords.words('english')
    filtered_words = [word for word in words if word not in stop_words]
    
    porter = PorterStemmer()
    stemmed = [porter.stem(word) for word in filtered_words]
    
    return " ".join(filtered_words) #convert stemmed words to 

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\maxim\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [10]:
df

,cat,username,text,date,geo,hashtags,tweet_id,mentions,permalink,replies,retweets,replies_to,mentioned_urls
0,before_pandemic,1baldchick,big wet smooshy kisses to you!!! i love u dear!!,2020-02-28 19:46:02+00:00,NaN,NaN,1233478483939971072,NaN,https://twitter.com/1baldchick/status/12334784...,0,0,amarcadia,NaN
1,before_pandemic,1baldchick,list of meds and check it for drug interaction...,2020-02-28 19:45:26+00:00,NaN,NaN,1233478333511479299,NaN,https://twitter.com/1baldchick/status/12334783...,1,0,1baldchick,NaN
2,before_pandemic,1baldchick,gab has loads of side effects. i always try to...,2020-02-28 19:44:18+00:00,NaN,NaN,1233478047212408834,NaN,https://twitter.com/1baldchick/status/12334780...,1,0,PajamaDaze,http://drugs.com
3,before_pandemic,1baldchick,happy friday to u good sir!! and to your littl...,2020-02-28 19:42:32+00:00,NaN,NaN,1233477603866050560,NaN,https://twitter.com/1baldchick/status/12334776...,0,0,ChronicPainDad,NaN
4,before_pandemic,1baldchick,this song says it all!,2020-02-28 08:34:04+00:00,NaN,NaN,1233309379497283584,NaN,https://twitter.com/1baldchick/status/12333093...,0,0,halsey,https://twitter.com/halsey/status/123326171381...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
270,since_pandemic,_tomdalton,total bullshit narrative.,2020-03-07 03:27:53+00:00,NaN,NaN,1236131428892651520,NaN,https://twitter.com/_tomdalton/status/12361314...,0,0,germanrlopez,NaN
271,since_pandemic,_tomdalton,doctors are not always right either and many w...,2020-03-05 23:36:05+00:00,NaN,NaN,1235710707993374721,NaN,https://twitter.com/_tomdalton/status/12357107...,0,1,StSenka,NaN
272,since_pandemic,_tomdalton,yes x1000,2020-03-05 18:57:13+00:00,NaN,NaN,1235640525740552192,NaN,https://twitter.com/_tomdalton/status/12356405...,0,1,TruthPharm,https://twitter.com/TruthPharm/status/12355996...
273,since_pandemic,_tomdalton,very important thread. already hearing the anx...,2020-03-05 18:55:54+00:00,NaN,NaN,1235640197926264832,@MorganGodvin,https://twitter.com/_tomdalton/status/12356401...,1,1,MorganGodvin,https://twitter.com/MorganGodvin/status/123561...


In [11]:
# #fix text
# for user in tqdm(list(before.keys())):
#     before[user]["text"] = np.vectorize(decodeAndClean)(before[user]["text"])

# for user in tqdm(list(after.keys())):
#     after[user]["text"] = np.vectorize(decodeAndClean)(after[user]["text"])

df["text"] = np.vectorize(decodeAndClean)(df["text"])

In [12]:
#nltk style text cleaning
df["text_nlp"] = np.vectorize(preprocess)(df["text"])

In [13]:
# apply vader sentiment
# for user in tqdm(list(before.keys())):
#     before[user]["sent"] = np.vectorize(getSentiment)(before[user]["text"])

# for user in tqdm(list(after.keys())):
#     after[user]["sent"] = np.vectorize(getSentiment)(after[user]["text"])

df["sent"] = np.vectorize(getSentiment)(df["text_nlp"])

In [14]:
# # apply emolex
# for user in tqdm(list(before.keys())):
#     before[user][["nrc_fear", 
#         "nrc_anger", 
#         "nrc_anticipation", 
#         "nrc_trust", 
#         "nrc_surprise", 
#         "nrc_positive", 
#         "nrc_negative", 
#         "nrc_sadness", 
#         "nrc_disgust", 
#         "nrc_joy"]] = list(map(pandasMultiCategorySentiment, before[user]["text"])) #np.vectorize(pandasMultiCategorySentiment)(before[user]["text"])

# for user in tqdm(list(after.keys())):
#     after[user][["nrc_fear", 
#         "nrc_anger", 
#         "nrc_anticipation", 
#         "nrc_trust", 
#         "nrc_surprise", 
#         "nrc_positive", 
#         "nrc_negative", 
#         "nrc_sadness", 
#         "nrc_disgust", 
#         "nrc_joy"]] = list(map(pandasMultiCategorySentiment, after[user]["text"])) #np.vectorize(pandasMultiCategorySentiment)(after[user]["text"])

df[["nrc_fear", 
    "nrc_anger", 
    "nrc_anticipation", 
    "nrc_trust", 
    "nrc_surprise", 
    "nrc_positive", 
    "nrc_negative", 
    "nrc_sadness", 
    "nrc_disgust", 
    "nrc_joy"]] = list(map(pandasMultiCategorySentiment, df["text_nlp"]))

In [15]:
# getSentiment(before["1baldchick"]["text"][0])
# list(map(getSentiment, before["1baldchick"]["text"]))
# before["1baldchick"].drop(before["1baldchick"][pd.isnull(before["1baldchick"]["text"])].index)
# before[file.stem]before["1baldchick"]

In [16]:
df

,cat,username,text,date,geo,hashtags,tweet_id,mentions,permalink,replies,...,nrc_fear,nrc_anger,nrc_anticipation,nrc_trust,nrc_surprise,nrc_positive,nrc_negative,nrc_sadness,nrc_disgust,nrc_joy
0,before_pandemic,1baldchick,big wet smooshy kisses to you!!! i love u dear!!,2020-02-28 19:46:02+00:00,NaN,NaN,1233478483939971072,NaN,https://twitter.com/1baldchick/status/12334784...,0,...,0.096774,0.096774,0.096774,0.129032,0.096774,0.096774,0.096774,0.096774,0.096774,0.096774
1,before_pandemic,1baldchick,list of meds and check it for drug interaction...,2020-02-28 19:45:26+00:00,NaN,NaN,1233478333511479299,NaN,https://twitter.com/1baldchick/status/12334783...,1,...,0.102941,0.117647,0.132353,0.088235,0.088235,0.088235,0.088235,0.102941,0.088235,0.102941
2,before_pandemic,1baldchick,gab has loads of side effects. i always try to...,2020-02-28 19:44:18+00:00,NaN,NaN,1233478047212408834,NaN,https://twitter.com/1baldchick/status/12334780...,1,...,0.100000,0.100000,0.106250,0.093750,0.100000,0.100000,0.100000,0.106250,0.100000,0.093750
3,before_pandemic,1baldchick,happy friday to u good sir!! and to your littl...,2020-02-28 19:42:32+00:00,NaN,NaN,1233477603866050560,NaN,https://twitter.com/1baldchick/status/12334776...,0,...,0.108108,0.108108,0.108108,0.108108,0.081081,0.081081,0.108108,0.108108,0.081081,0.108108
4,before_pandemic,1baldchick,this song says it all!,2020-02-28 08:34:04+00:00,NaN,NaN,1233309379497283584,NaN,https://twitter.com/1baldchick/status/12333093...,0,...,0.125000,0.125000,0.125000,0.125000,0.000000,0.125000,0.125000,0.125000,0.000000,0.125000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
270,since_pandemic,_tomdalton,total bullshit narrative.,2020-03-07 03:27:53+00:00,NaN,NaN,1236131428892651520,NaN,https://twitter.com/_tomdalton/status/12361314...,0,...,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000
271,since_pandemic,_tomdalton,doctors are not always right either and many w...,2020-03-05 23:36:05+00:00,NaN,NaN,1235710707993374721,NaN,https://twitter.com/_tomdalton/status/12357107...,0,...,0.098592,0.084507,0.098592,0.098592,0.112676,0.084507,0.112676,0.126761,0.084507,0.098592
272,since_pandemic,_tomdalton,yes x1000,2020-03-05 18:57:13+00:00,NaN,NaN,1235640525740552192,NaN,https://twitter.com/_tomdalton/status/12356405...,0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
273,since_pandemic,_tomdalton,very important thread. already hearing the anx...,2020-03-05 18:55:54+00:00,NaN,NaN,1235640197926264832,@MorganGodvin,https://twitter.com/_tomdalton/status/12356401...,1,...,0.116279,0.116279,0.116279,0.046512,0.116279,0.093023,0.116279,0.093023,0.069767,0.116279


In [17]:
df.to_csv("proc.csv")